In [ ]:
# Importar librerías
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import joblib

In [ ]:
# Dataset sintético (reemplazar con dataset real en producción)
rng = np.random.default_rng(42)
n = 500
df = pd.DataFrame({
    'tiempo_contrato_meses': rng.integers(0, 36, size=n),
    'retrasos_pago': rng.integers(0, 6, size=n),
    'uso_mensual': rng.normal(12, 4, size=n).clip(0),
    'plan': rng.choice(['Basic','Standard','Premium'], size=n)
})
# Regla para probabilidad de churn
z = -1.0 + 0.08*df['retrasos_pago'] - 0.03*df['tiempo_contrato_meses'] - 0.02*df['uso_mensual'] + df['plan'].map({'Basic':0.15,'Standard':0.10,'Premium':0.05})
p = 1/(1+np.exp(-z))
df['churn'] = (p >= 0.5).astype(int)
df.head()

In [ ]:
# One-hot del plan
df_ml = pd.get_dummies(df, columns=['plan'], drop_first=True)
X = df_ml.drop(columns=['churn'])
y = df_ml['churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

In [ ]:
# Pipeline: escalado + regresión logística
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=1000))
])
pipe.fit(X_train, y_train)
print(classification_report(y_test, pipe.predict(X_test)))

In [ ]:
# Guardar artefactos
joblib.dump(pipe, 'churn_pipeline.pkl')
joblib.dump(list(X.columns), 'feature_names.pkl')
print('Artefactos guardados: churn_pipeline.pkl, feature_names.pkl')

## Contrato de integración (JSON)
Entrada:
{
"tiempo_contrato_meses": 12,
"retrasos_pago": 2,
"uso_mensual": 14.5,
"plan": "Premium"
}

Salida:
{ "prevision": "Va a cancelar", "probabilidad": 0.81 }

## Métricas del modelo (Accuracy, Precision, Recall, F1)
Este bloque calcula métricas de clasificación para el modelo entrenado y muestra el reporte de clasificación y la matriz de confusión. Asegúrate de que existan las variables `model`, `X_test` y `y_test` (producidas por tu pipeline de entrenamiento).

In [ ]:
# Métricas del modelo
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
)

# Verificaciones mínimas de variables requeridas
try:
    _ = (model, X_test, y_test)
except NameError as e:
    raise NameError("Variables esperadas no existen: defina 'model', 'X_test', 'y_test' antes de ejecutar métricas.") from e

# Predicción del conjunto de prueba
y_pred = model.predict(X_test)

# Métricas agregadas (weighted para evitar errores si las etiquetas no son binarias 0/1)
accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision (weighted): {precision:.4f}")
print(f"Recall (weighted): {recall:.4f}")
print(f"F1-score (weighted): {f1:.4f}")

# Reporte detallado por clase
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title("Matriz de Confusión")
plt.show()